## LFCC + CNN

In [1]:
"""
Block 1: Dataset Preparation and Train-Test Split
This block handles loading the dataset, creating a balanced subset, and splitting into train/test sets.
"""

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import shutil

# Configuration
DATASET_SIZE = 136000  # 68,000 real + 68,000 AI
TEST_SIZE = 0.2
RANDOM_STATE = 42
BASE_PATH = '/kaggle/input/asins-voice/build'
REAL_CLIPS_PATH = f'{BASE_PATH}/clips'
AI_CLIPS_PATH = f'{BASE_PATH}/clips_AI'

print("=" * 60)
print("BLOCK 1: DATASET PREPARATION")
print("=" * 60)

# Step 1: Collect all audio files
print("\n[Step 1/5] Collecting audio files...")

real_files = []
ai_files = []

for file in os.listdir(REAL_CLIPS_PATH):
    if file.endswith(('.mp3', '.wav', '.ogg', '.flac')):
        real_files.append(file)

for file in os.listdir(AI_CLIPS_PATH):
    if file.endswith(('.mp3', '.wav', '.ogg', '.flac')):
        ai_files.append(file)

print(f"Found {len(real_files)} real audio files")
print(f"Found {len(ai_files)} AI audio files")

# Step 2: Create matched pairs (same transcript)
print("\n[Step 2/5] Creating matched pairs...")

# Get intersection of filenames to ensure we have matching pairs
real_set = set(real_files)
ai_set = set(ai_files)
matched_files = list(real_set.intersection(ai_set))

print(f"Found {len(matched_files)} matched pairs")

# Step 3: Sample balanced subset
print("\n[Step 3/5] Sampling balanced subset...")

np.random.seed(RANDOM_STATE)
samples_per_class = DATASET_SIZE // 2

if len(matched_files) < samples_per_class:
    print(f"WARNING: Only {len(matched_files)} matched pairs available, using all of them")
    samples_per_class = len(matched_files)
    
selected_files = np.random.choice(matched_files, size=samples_per_class, replace=False)

# Step 4: Create dataframe
print("\n[Step 4/5] Creating dataset dataframe...")

data = []
for filename in selected_files:
    # Real audio
    data.append({
        'filename': filename,
        'filepath': os.path.join(REAL_CLIPS_PATH, filename),
        'label': 0,  # 0 for REAL
        'label_text': 'REAL'
    })
    
    # AI audio
    data.append({
        'filename': filename,
        'filepath': os.path.join(AI_CLIPS_PATH, filename),
        'label': 1,  # 1 for AI_GENERATED
        'label_text': 'AI_GENERATED'
    })

df = pd.DataFrame(data)

# Shuffle the dataframe
df = df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

print(f"Total samples: {len(df)}")
print(f"\nClass distribution:")
print(df['label_text'].value_counts())

# Step 5: Train-test split
print("\n[Step 5/5] Creating train-test split...")

train_df, test_df = train_test_split(
    df, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=df['label']
)

print(f"\nTrain set: {len(train_df)} samples")
print(f"Test set: {len(test_df)} samples")
print(f"\nTrain class distribution:")
print(train_df['label_text'].value_counts())
print(f"\nTest class distribution:")
print(test_df['label_text'].value_counts())

# Save dataframes for later use
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

print("\n✓ Dataset preparation complete!")
print("✓ Saved train_data.csv and test_data.csv")
print("=" * 60)

BLOCK 1: DATASET PREPARATION

[Step 1/5] Collecting audio files...
Found 68030 real audio files
Found 68030 AI audio files

[Step 2/5] Creating matched pairs...
Found 68030 matched pairs

[Step 3/5] Sampling balanced subset...

[Step 4/5] Creating dataset dataframe...
Total samples: 136000

Class distribution:
label_text
REAL            68000
AI_GENERATED    68000
Name: count, dtype: int64

[Step 5/5] Creating train-test split...

Train set: 108800 samples
Test set: 27200 samples

Train class distribution:
label_text
AI_GENERATED    54400
REAL            54400
Name: count, dtype: int64

Test class distribution:
label_text
REAL            13600
AI_GENERATED    13600
Name: count, dtype: int64

✓ Dataset preparation complete!
✓ Saved train_data.csv and test_data.csv


In [ ]:
"""
Block 2 MEMORY-EFFICIENT: Feature Extraction with Proper Memory Management
FIXES: Kernel death due to memory overflow

Key changes:
- Save features incrementally to disk (batches of 1000)
- Clear memory after each batch
- Use memory-mapped arrays for large datasets
- Explicit garbage collection
"""

import numpy as np
import librosa
import pandas as pd
from tqdm import tqdm
import gc
import warnings
import os
warnings.filterwarnings('ignore')

print("=" * 80)
print("BLOCK 2 MEMORY-EFFICIENT: FEATURE EXTRACTION")
print("=" * 80)

# Configuration
SAMPLE_RATE = 16000
N_LFCC = 40
N_FFT = 2048
HOP_LENGTH = 512
MAX_DURATION = 10
BATCH_SIZE = 100 
SAVE_EVERY = 1000  # Save to disk every 1000 samples

def extract_lfcc(audio_path):
    """Extract LFCC features from audio file."""
    try:
        y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=MAX_DURATION)
        
        lfcc = librosa.feature.mfcc(
            y=y, 
            sr=sr, 
            n_mfcc=N_LFCC,
            n_fft=N_FFT,
            hop_length=HOP_LENGTH
        )
        
        target_length = int(MAX_DURATION * sr / HOP_LENGTH)
        
        if lfcc.shape[1] < target_length:
            pad_width = target_length - lfcc.shape[1]
            lfcc = np.pad(lfcc, ((0, 0), (0, pad_width)), mode='constant')
        else:
            lfcc = lfcc[:, :target_length]
        
        return lfcc
    
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        target_length = int(MAX_DURATION * SAMPLE_RATE / HOP_LENGTH)
        return np.zeros((N_LFCC, target_length))

def extract_features_memory_efficient(df, output_path, batch_size=BATCH_SIZE):
    """
    Extract features with memory-efficient processing.
    Saves features to disk incrementally to avoid memory overflow.
    """
    total_samples = len(df)
    num_batches = (total_samples + batch_size - 1) // batch_size
    
    # Create temporary directory for partial saves
    temp_dir = 'temp_features'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Determine feature shape from first sample
    print("Determining feature shape from first sample...")
    first_feature = extract_lfcc(df.iloc[0]['filepath'])
    feature_shape = first_feature.shape
    print(f"Feature shape: {feature_shape}")
    
    # Process in batches and save incrementally
    chunk_files = []
    current_chunk = []
    current_labels = []
    current_filenames = []
    chunk_idx = 0
    
    print(f"\nProcessing {total_samples} samples in {num_batches} batches...")
    print(f"Saving to disk every {SAVE_EVERY} samples to prevent memory overflow...")
    
    for batch_idx in tqdm(range(num_batches), desc="Processing batches"):
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, total_samples)
        batch_df = df.iloc[batch_start:batch_end]
        
        # Process batch
        for idx, row in batch_df.iterrows():
            lfcc = extract_lfcc(row['filepath'])
            current_chunk.append(lfcc)
            current_labels.append(row['label'])
            current_filenames.append(row['filename'])
            
            # Save chunk when reaching SAVE_EVERY samples
            if len(current_chunk) >= SAVE_EVERY:
                chunk_file = os.path.join(temp_dir, f'chunk_{chunk_idx}.npz')
                np.savez_compressed(
                    chunk_file,
                    X=np.array(current_chunk),
                    y=np.array(current_labels),
                    filenames=current_filenames
                )
                chunk_files.append(chunk_file)
                chunk_idx += 1
                
                # Clear memory
                current_chunk = []
                current_labels = []
                current_filenames = []
                gc.collect()
                
                print(f"\n  Saved chunk {chunk_idx} to disk, cleared memory")
        
        # Clear memory after each batch
        if (batch_idx + 1) % 10 == 0:
            gc.collect()
    
    # Save remaining samples
    if len(current_chunk) > 0:
        chunk_file = os.path.join(temp_dir, f'chunk_{chunk_idx}.npz')
        np.savez_compressed(
            chunk_file,
            X=np.array(current_chunk),
            y=np.array(current_labels),
            filenames=current_filenames
        )
        chunk_files.append(chunk_file)
        chunk_idx += 1
        print(f"\n  Saved final chunk {chunk_idx}")
    
    # Combine chunks into final file
    print(f"\nCombining {len(chunk_files)} chunks into final file...")
    
    all_features = []
    all_labels = []
    all_filenames = []
    
    for chunk_file in tqdm(chunk_files, desc="Loading chunks"):
        chunk_data = np.load(chunk_file, allow_pickle=True)
        all_features.append(chunk_data['X'])
        all_labels.append(chunk_data['y'])
        all_filenames.extend(chunk_data['filenames'])
        
        # Delete chunk file to save disk space
        os.remove(chunk_file)
    
    # Concatenate all features
    X = np.concatenate(all_features, axis=0)
    y = np.concatenate(all_labels, axis=0)
    
    # Save final file
    print(f"Saving final features to {output_path}...")
    np.savez_compressed(output_path, X=X, y=y, filenames=all_filenames)
    
    # Cleanup
    os.rmdir(temp_dir)
    
    # Clear memory
    del all_features, all_labels, X, y
    gc.collect()
    
    print(f"✓ Saved {output_path}")
    
    return output_path

# Load datasets
print("\n[Step 1/3] Loading dataset splits...")
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")

# Extract training features
print("\n[Step 2/3] Extracting TRAINING features (memory-efficient)...")
extract_features_memory_efficient(
    train_df, 
    'features_train.npz',
    batch_size=BATCH_SIZE
)

# Force garbage collection before test set
print("\nClearing memory before processing test set...")
gc.collect()

# Extract test features
print("\n[Step 3/3] Extracting TEST features (memory-efficient)...")
extract_features_memory_efficient(
    test_df, 
    'features_test.npz',
    batch_size=BATCH_SIZE
)

# Final summary
print("\n" + "=" * 80)
print("MEMORY-EFFICIENT FEATURE EXTRACTION COMPLETE!")
print("=" * 80)

# Load and verify
train_data = np.load('features_train.npz', allow_pickle=True)
test_data = np.load('features_test.npz', allow_pickle=True)

print(f"\nVerification:")
print(f"  Train features: {train_data['X'].shape}")
print(f"  Train labels: {train_data['y'].shape}")
print(f"  Test features: {test_data['X'].shape}")
print(f"  Test labels: {test_data['y'].shape}")

print(f"\n✓ All features extracted successfully!")
print(f"✓ Memory management: Incremental saving prevented overflow")
print(f"✓ No kernel death! 🎉")
print("=" * 80)

# Clear everything
del train_data, test_data
gc.collect()

BLOCK 2 MEMORY-EFFICIENT: FEATURE EXTRACTION

[Step 1/3] Loading dataset splits...
Train samples: 108800
Test samples: 27200

[Step 2/3] Extracting TRAINING features (memory-efficient)...
Determining feature shape from first sample...
Feature shape: (40, 312)

Processing 108800 samples in 2176 batches...
Saving to disk every 1000 samples to prevent memory overflow...


Processing batches:   1%|          | 20/2176 [00:25<59:51,  1.67s/it]


  Saved chunk 1 to disk, cleared memory


Processing batches:   2%|▏         | 40/2176 [00:51<1:02:13,  1.75s/it]


  Saved chunk 2 to disk, cleared memory


...

Processing batches: 100%|██████████| 544/544 [10:49<00:00,  1.19s/it]



  Saved final chunk 28

Combining 28 chunks into final file...


Loading chunks: 100%|██████████| 28/28 [00:05<00:00,  4.71it/s]


Saving final features to features_test.npz...
✓ Saved features_test.npz

MEMORY-EFFICIENT FEATURE EXTRACTION COMPLETE!

Verification:
  Train features: (108800, 40, 312)
  Train labels: (108800,)
  Test features: (27200, 40, 312)
  Test labels: (27200,)

✓ All features extracted successfully!
✓ Memory management: Incremental saving prevented overflow
✓ No kernel death! 🎉


44

In [3]:
"""
Block 3: CNN Model Architecture and Training
Implements a CNN for binary classification with checkpoint management.
"""

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, callbacks
from sklearn.metrics import classification_report, confusion_matrix
import gc

print("=" * 60)
print("BLOCK 3: CNN MODEL TRAINING")
print("=" * 60)

# Check GPU availability
print("\n[GPU Check]")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Model configuration
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
PATIENCE = 5

# Load features
print("\n[Step 1/5] Loading features...")
train_data = np.load('features_train.npz', allow_pickle=True)
test_data = np.load('features_test.npz', allow_pickle=True)

X_train = train_data['X']
y_train = train_data['y']
X_test = test_data['X']
y_test = test_data['y']

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

# Reshape for CNN: (samples, height, width, channels)
# LFCC features: (n_lfcc, time_steps) -> (n_lfcc, time_steps, 1)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print(f"Reshaped train: {X_train.shape}")
print(f"Reshaped test: {X_test.shape}")

# Normalize features
print("\n[Step 2/5] Normalizing features...")
mean = X_train.mean()
std = X_train.std()
X_train = (X_train - mean) / (std + 1e-8)
X_test = (X_test - mean) / (std + 1e-8)

# Save normalization parameters
np.savez('normalization_params.npz', mean=mean, std=std)

# Build CNN model
print("\n[Step 3/5] Building CNN model...")

def build_cnn_model(input_shape):
    """
    CNN architecture optimized for audio classification.
    """
    inputs = layers.Input(shape=input_shape, name='input')
    
    # Block 1
    x = layers.Conv2D(32, (3, 3), padding='same', name='conv1')(inputs)
    x = layers.BatchNormalization(name='bn1')(x)
    x = layers.Activation('relu', name='relu1')(x)
    x = layers.MaxPooling2D((2, 2), name='pool1')(x)
    x = layers.Dropout(0.25, name='dropout1')(x)
    
    # Block 2
    x = layers.Conv2D(64, (3, 3), padding='same', name='conv2')(x)
    x = layers.BatchNormalization(name='bn2')(x)
    x = layers.Activation('relu', name='relu2')(x)
    x = layers.MaxPooling2D((2, 2), name='pool2')(x)
    x = layers.Dropout(0.25, name='dropout2')(x)
    
    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same', name='conv3')(x)
    x = layers.BatchNormalization(name='bn3')(x)
    x = layers.Activation('relu', name='relu3')(x)
    x = layers.MaxPooling2D((2, 2), name='pool3')(x)
    x = layers.Dropout(0.25, name='dropout3')(x)
    
    # Block 4
    x = layers.Conv2D(256, (3, 3), padding='same', name='conv4')(x)
    x = layers.BatchNormalization(name='bn4')(x)
    x = layers.Activation('relu', name='relu4')(x)
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    
    # Dense layers
    x = layers.Dense(128, activation='relu', name='dense1')(x)
    x = layers.Dropout(0.5, name='dropout4')(x)
    x = layers.Dense(64, activation='relu', name='dense2')(x)
    x = layers.Dropout(0.5, name='dropout5')(x)
    
    # Output
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='VoiceClassifierCNN')
    
    return model

# Create model
input_shape = X_train.shape[1:]  # (n_lfcc, time_steps, 1)
model = build_cnn_model(input_shape)

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', 
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')]
)

model.summary()

# Setup callbacks for checkpoint management
print("\n[Step 4/5] Setting up callbacks...")

# Create checkpoint directory
import os
os.makedirs('checkpoints', exist_ok=True)

# Callback: Save best model only
checkpoint_best = callbacks.ModelCheckpoint(
    filepath='checkpoints/best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Callback: Save latest checkpoint (overwrite)
checkpoint_latest = callbacks.ModelCheckpoint(
    filepath='checkpoints/latest_checkpoint.h5',
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    mode='min',
    verbose=0
)

# Callback: Early stopping
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True,
    verbose=1
)

# Callback: Reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Callback: CSV logger
csv_logger = callbacks.CSVLogger('training_log.csv', append=False)

callback_list = [
    checkpoint_best,
    checkpoint_latest,
    early_stop,
    reduce_lr,
    csv_logger
]

# Train model
print("\n[Step 5/5] Training model...")
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"Early stopping patience: {PATIENCE}")

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=callback_list,
    verbose=1
)

print("\n✓ Model training complete!")
print("✓ Best model saved to: checkpoints/best_model.h5")
print("✓ Latest checkpoint saved to: checkpoints/latest_checkpoint.h5")
print("✓ Training log saved to: training_log.csv")
print("=" * 60)

# Evaluate on test set
print("\n[Evaluation on Test Set]")
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

# Generate predictions
y_pred_proba = model.predict(X_test, batch_size=BATCH_SIZE, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Classification report
print("\n[Classification Report]")
print(classification_report(y_test, y_pred, 
                          target_names=['REAL', 'AI_GENERATED'],
                          digits=4))

# Confusion matrix
print("\n[Confusion Matrix]")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"\nTrue Negatives (REAL correctly classified): {cm[0][0]}")
print(f"False Positives (REAL classified as AI): {cm[0][1]}")
print(f"False Negatives (AI classified as REAL): {cm[1][0]}")
print(f"True Positives (AI correctly classified): {cm[1][1]}")

# Clear memory
del X_train, y_train, X_test, y_test
gc.collect()

print("\n" + "=" * 60)

2026-02-10 02:59:12.989868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770692353.132334      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770692353.171461      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770692353.500235      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770692353.500258      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770692353.500261      23 computation_placer.cc:177] computation placer alr

BLOCK 3: CNN MODEL TRAINING

[GPU Check]
TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

[Step 1/5] Loading features...
Train shape: (108800, 40, 312)
Test shape: (27200, 40, 312)
Reshaped train: (108800, 40, 312, 1)
Reshaped test: (27200, 40, 312, 1)

[Step 2/5] Normalizing features...

[Step 3/5] Building CNN model...


I0000 00:00:1770692401.024574      23 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770692401.030635      23 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "VoiceClassifierCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 40, 312, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 40, 312, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 40, 312, 32)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1 (Activation)              │ (None, 40, 312, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 20, 156, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 20, 156, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 20, 156, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 20, 156, 64)    │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2 (Activation)              │ (None, 20, 156, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 10, 78, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 10, 78, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 10, 78, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 10, 78, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu3 (Activation)              │ (None, 10, 78, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 5, 39, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 5, 39, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 5, 39, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4 (BatchNormalization)        │ (None, 5, 39, 256)     │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu4 (Activation)              │ (None, 5, 39, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling2D)    │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout4 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout5 (Dropout)              │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            6

...

3400/3400 ━━━━━━━━━━━━━━━━━━━━ 73s 22ms/step - accuracy: 0.9998 - loss: 8.1567e-04 - precision: 0.9998 - recall: 0.9998 - val_accuracy: 0.9997 - val_loss: 7.2076e-04 - val_precision: 0.9995 - val_recall: 1.0000 - learning_rate: 5.0000e-04
Epoch 27/50
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9997 - loss: 0.0012 - precision: 0.9996 - recall: 0.9997
Epoch 27: val_loss did not improve from 0.00026


3400/3400 ━━━━━━━━━━━━━━━━━━━━ 74s 22ms/step - accuracy: 0.9997 - loss: 0.0012 - precision: 0.9996 - recall: 0.9997 - val_accuracy: 0.9999 - val_loss: 6.2631e-04 - val_precision: 0.9999 - val_recall: 0.9999 - learning_rate: 5.0000e-04
Epoch 28/50
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9998 - loss: 6.6239e-04 - precision: 0.9998 - recall: 0.9998
Epoch 28: val_loss did not improve from 0.00026


3400/3400 ━━━━━━━━━━━━━━━━━━━━ 73s 22ms/step - accuracy: 0.9998 - loss: 6.6247e-04 - precision: 0.9998 - recall: 0.9998 - val_accuracy: 0.9999 - val_loss: 2.9681e-04 - val_precision: 0.9999 - val_recall: 0.9999 - learning_rate: 5.0000e-04
Epoch 29/50
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9998 - loss: 8.6876e-04 - precision: 0.9997 - recall: 0.9999
Epoch 29: val_loss did not improve from 0.00026


3400/3400 ━━━━━━━━━━━━━━━━━━━━ 74s 22ms/step - accuracy: 0.9998 - loss: 8.6883e-04 - precision: 0.9997 - recall: 0.9999 - val_accuracy: 0.9999 - val_loss: 6.7945e-04 - val_precision: 0.9997 - val_recall: 1.0000 - learning_rate: 5.0000e-04
Epoch 30/50
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9998 - loss: 7.7839e-04 - precision: 0.9998 - recall: 0.9999
Epoch 30: val_loss did not improve from 0.00026



Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 73s 22ms/step - accuracy: 0.9998 - loss: 7.7852e-04 - precision: 0.9998 - recall: 0.9999 - val_accuracy: 0.9995 - val_loss: 0.0013 - val_precision: 0.9990 - val_recall: 1.0000 - learning_rate: 5.0000e-04
Epoch 30: early stopping
Restoring model weights from the end of the best epoch: 25.

✓ Model training complete!
✓ Best model saved to: checkpoints/best_model.h5
✓ Latest checkpoint saved to: checkpoints/latest_checkpoint.h5
✓ Training log saved to: training_log.csv

[Evaluation on Test Set]
Test Loss: 0.0003
Test Accuracy: 0.9999
Test Precision: 0.9998
Test Recall: 1.0000

[Classification Report]
              precision    recall  f1-score   support

        REAL     1.0000    0.9998    0.9999     13600
AI_GENERATED     0.9998    1.0000    0.9999     13600

    accuracy                         0.9999     27200
   macro avg     0.9999    0.9999    0.9999     27200
weighted avg  

In [4]:
"""
Block 4: Rule-Based Explanation Engine
Analyzes audio features to generate human-readable explanations.
"""

import numpy as np
import librosa
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("BLOCK 4: EXPLANATION ENGINE")
print("=" * 60)

class AudioAnalyzer:
    """
    Analyzes audio files to extract interpretable features for explanations.
    """
    
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
    
    def analyze_audio(self, audio_path):
        """
        Extract various audio features for rule-based explanation.
        Returns a dictionary of analysis results.
        """
        try:
            # Load audio
            y, sr = librosa.load(audio_path, sr=self.sample_rate, duration=10)
            
            analysis = {}
            
            # 1. PITCH ANALYSIS
            pitch_features = self._analyze_pitch(y, sr)
            analysis.update(pitch_features)
            
            # 2. SILENCE ANALYSIS
            silence_features = self._analyze_silence(y, sr)
            analysis.update(silence_features)
            
            # 3. BREATH ANALYSIS
            breath_features = self._analyze_breaths(y, sr)
            analysis.update(breath_features)
            
            # 4. SPECTRAL ANALYSIS
            spectral_features = self._analyze_spectral(y, sr)
            analysis.update(spectral_features)
            
            # 5. ENERGY ANALYSIS
            energy_features = self._analyze_energy(y, sr)
            analysis.update(energy_features)
            
            return analysis
            
        except Exception as e:
            print(f"Error analyzing audio: {e}")
            return self._get_default_analysis()
    
    def _analyze_pitch(self, y, sr):
        """Analyze pitch characteristics."""
        # Extract pitch using piptrack
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        
        # Get pitch values with significant magnitude
        pitch_values = []
        for t in range(pitches.shape[1]):
            index = magnitudes[:, t].argmax()
            pitch = pitches[index, t]
            if pitch > 0:
                pitch_values.append(pitch)
        
        if len(pitch_values) == 0:
            return {
                'pitch_variance': 0,
                'pitch_consistency': 0,
                'pitch_range': 0
            }
        
        pitch_values = np.array(pitch_values)
        
        # Calculate pitch statistics
        pitch_variance = np.var(pitch_values)
        pitch_std = np.std(pitch_values)
        pitch_mean = np.mean(pitch_values)
        pitch_range = np.max(pitch_values) - np.min(pitch_values)
        
        # Consistency score (lower variance = more consistent/robotic)
        pitch_consistency = 1.0 / (1.0 + pitch_variance / 1000)
        
        return {
            'pitch_variance': float(pitch_variance),
            'pitch_consistency': float(pitch_consistency),
            'pitch_range': float(pitch_range),
            'pitch_mean': float(pitch_mean),
            'pitch_std': float(pitch_std)
        }
    
    def _analyze_silence(self, y, sr):
        """Analyze silence patterns."""
        # Split audio into frames
        frame_length = 2048
        hop_length = 512
        
        # Calculate RMS energy
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        
        # Define silence threshold (adaptive)
        silence_threshold = np.percentile(rms, 20)
        
        # Find silent frames
        silent_frames = rms < silence_threshold
        silence_ratio = np.mean(silent_frames)
        
        # Analyze silence intervals
        silence_intervals = self._get_intervals(silent_frames)
        
        if len(silence_intervals) > 0:
            avg_silence_duration = np.mean([end - start for start, end in silence_intervals])
            silence_uniformity = 1.0 - np.std([end - start for start, end in silence_intervals]) / (avg_silence_duration + 1e-8)
        else:
            avg_silence_duration = 0
            silence_uniformity = 0
        
        return {
            'silence_ratio': float(silence_ratio),
            'num_silence_intervals': len(silence_intervals),
            'avg_silence_duration': float(avg_silence_duration),
            'silence_uniformity': float(silence_uniformity)
        }
    
    def _analyze_breaths(self, y, sr):
        """Analyze breath-like sounds."""
        # Breaths typically have specific spectral characteristics
        # High frequency content, low amplitude, short duration
        
        # Calculate spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        
        # Calculate zero crossing rate (breaths have higher ZCR)
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        
        # Calculate RMS
        rms = librosa.feature.rms(y=y)[0]
        
        # Detect potential breath events
        # Breaths: high ZCR + low RMS + moderate spectral centroid
        rms_threshold = np.percentile(rms, 30)
        zcr_threshold = np.percentile(zcr, 70)
        
        potential_breaths = (zcr > zcr_threshold) & (rms < rms_threshold)
        
        breath_ratio = np.mean(potential_breaths)
        num_breath_events = len(self._get_intervals(potential_breaths))
        
        return {
            'breath_ratio': float(breath_ratio),
            'num_breath_events': num_breath_events,
            'avg_zcr': float(np.mean(zcr))
        }
    
    def _analyze_spectral(self, y, sr):
        """Analyze spectral characteristics."""
        # Spectral features
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        
        return {
            'spectral_centroid_mean': float(np.mean(spectral_centroid)),
            'spectral_centroid_std': float(np.std(spectral_centroid)),
            'spectral_rolloff_mean': float(np.mean(spectral_rolloff)),
            'spectral_bandwidth_mean': float(np.mean(spectral_bandwidth)),
            'spectral_bandwidth_std': float(np.std(spectral_bandwidth))
        }
    
    def _analyze_energy(self, y, sr):
        """Analyze energy patterns."""
        rms = librosa.feature.rms(y=y)[0]
        
        # Energy statistics
        energy_mean = np.mean(rms)
        energy_std = np.std(rms)
        energy_variance = np.var(rms)
        
        # Energy consistency (AI voices tend to have more consistent energy)
        energy_consistency = 1.0 / (1.0 + energy_variance * 100)
        
        return {
            'energy_mean': float(energy_mean),
            'energy_std': float(energy_std),
            'energy_variance': float(energy_variance),
            'energy_consistency': float(energy_consistency)
        }
    
    def _get_intervals(self, boolean_array):
        """Get start and end indices of True intervals."""
        intervals = []
        in_interval = False
        start = 0
        
        for i, val in enumerate(boolean_array):
            if val and not in_interval:
                start = i
                in_interval = True
            elif not val and in_interval:
                intervals.append((start, i))
                in_interval = False
        
        if in_interval:
            intervals.append((start, len(boolean_array)))
        
        return intervals
    
    def _get_default_analysis(self):
        """Return default analysis in case of error."""
        return {
            'pitch_variance': 0,
            'pitch_consistency': 0,
            'pitch_range': 0,
            'silence_ratio': 0,
            'num_silence_intervals': 0,
            'avg_silence_duration': 0,
            'silence_uniformity': 0,
            'breath_ratio': 0,
            'num_breath_events': 0,
            'avg_zcr': 0,
            'spectral_centroid_mean': 0,
            'energy_consistency': 0
        }


class ExplanationGenerator:
    """
    Generates human-readable explanations based on audio analysis and prediction.
    """
    
    def __init__(self):
        self.analyzer = AudioAnalyzer()
    
    def generate_explanation(self, audio_path, prediction_class, confidence_score):
        """
        Generate explanation based on audio analysis.
        
        Args:
            audio_path: Path to audio file
            prediction_class: 'REAL' or 'AI_GENERATED'
            confidence_score: Confidence score (0-1)
        
        Returns:
            Explanation string
        """
        # Analyze audio
        analysis = self.analyzer.analyze_audio(audio_path)
        
        # Generate explanation based on class and features
        if prediction_class == 'AI_GENERATED':
            explanation = self._explain_ai_generated(analysis, confidence_score)
        else:
            explanation = self._explain_real(analysis, confidence_score)
        
        return explanation
    
    def _explain_ai_generated(self, analysis, confidence):
        """Generate explanation for AI-generated classification."""
        reasons = []
        
        # Check pitch consistency (AI tends to be more consistent)
        if analysis['pitch_consistency'] > 0.7:
            reasons.append("unnatural pitch consistency")
        
        # Check pitch variance (AI tends to have lower variance)
        if analysis['pitch_variance'] < 500:
            reasons.append("limited pitch variation")
        
        # Check silence patterns (AI tends to have uniform silences)
        if analysis['silence_uniformity'] > 0.7:
            reasons.append("uniform silence patterns")
        
        # Check breath events (AI tends to lack natural breaths)
        if analysis['num_breath_events'] < 2:
            reasons.append("absence of natural breath sounds")
        
        # Check energy consistency (AI tends to be more consistent)
        if analysis['energy_consistency'] > 0.7:
            reasons.append("robotic energy patterns")
        
        # Check spectral characteristics
        if analysis['spectral_bandwidth_std'] < 200:
            reasons.append("limited spectral variation")
        
        # Construct explanation
        if len(reasons) == 0:
            explanation = "Artificial speech patterns detected"
        elif len(reasons) == 1:
            explanation = f"{reasons[0].capitalize()} detected"
        elif len(reasons) == 2:
            explanation = f"{reasons[0].capitalize()} and {reasons[1]} detected"
        else:
            explanation = f"{reasons[0].capitalize()}, {reasons[1]}, and {reasons[2]} detected"
        
        return explanation
    
    def _explain_real(self, analysis, confidence):
        """Generate explanation for real voice classification."""
        reasons = []
        
        # Check pitch variation (human tends to have more variation)
        if analysis['pitch_variance'] > 1000:
            reasons.append("natural pitch variation")
        
        # Check breath events (humans have breath sounds)
        if analysis['num_breath_events'] >= 2:
            reasons.append("natural breath patterns")
        
        # Check energy patterns (humans have less consistent energy)
        if analysis['energy_consistency'] < 0.5:
            reasons.append("organic energy fluctuations")
        
        # Check silence patterns (humans have varied pauses)
        if analysis['silence_uniformity'] < 0.5:
            reasons.append("natural pause patterns")
        
        # Check spectral variation
        if analysis['spectral_bandwidth_std'] > 300:
            reasons.append("rich spectral dynamics")
        
        # Construct explanation
        if len(reasons) == 0:
            explanation = "Human speech characteristics detected"
        elif len(reasons) == 1:
            explanation = f"{reasons[0].capitalize()} detected"
        elif len(reasons) == 2:
            explanation = f"{reasons[0].capitalize()} and {reasons[1]} detected"
        else:
            explanation = f"{reasons[0].capitalize()}, {reasons[1]}, and {reasons[2]} detected"
        
        return explanation


# Test the explanation engine
print("\n[Testing Explanation Engine]")

analyzer = AudioAnalyzer()
explainer = ExplanationGenerator()

print("✓ AudioAnalyzer initialized")
print("✓ ExplanationGenerator initialized")
print("\nReady to generate explanations for predictions!")
print("=" * 60)

BLOCK 4: EXPLANATION ENGINE

[Testing Explanation Engine]
✓ AudioAnalyzer initialized
✓ ExplanationGenerator initialized

Ready to generate explanations for predictions!


In [5]:
"""
Block 5: Language Detection and Inference Pipeline (FIXED - All-in-One)
Complete pipeline for making predictions with language detection and explanations.
All dependencies included - no imports from other blocks needed.
"""

import numpy as np
import librosa
import json
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("BLOCK 5: INFERENCE PIPELINE")
print("=" * 60)

# ============================================================================
# AudioAnalyzer Class (from Block 4)
# ============================================================================

class AudioAnalyzer:
    """
    Analyzes audio files to extract interpretable features for explanations.
    """
    
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
    
    def analyze_audio(self, audio_path):
        """
        Extract various audio features for rule-based explanation.
        Returns a dictionary of analysis results.
        """
        try:
            # Load audio
            y, sr = librosa.load(audio_path, sr=self.sample_rate, duration=10)
            
            analysis = {}
            
            # 1. PITCH ANALYSIS
            pitch_features = self._analyze_pitch(y, sr)
            analysis.update(pitch_features)
            
            # 2. SILENCE ANALYSIS
            silence_features = self._analyze_silence(y, sr)
            analysis.update(silence_features)
            
            # 3. BREATH ANALYSIS
            breath_features = self._analyze_breaths(y, sr)
            analysis.update(breath_features)
            
            # 4. SPECTRAL ANALYSIS
            spectral_features = self._analyze_spectral(y, sr)
            analysis.update(spectral_features)
            
            # 5. ENERGY ANALYSIS
            energy_features = self._analyze_energy(y, sr)
            analysis.update(energy_features)
            
            return analysis
            
        except Exception as e:
            print(f"Error analyzing audio: {e}")
            return self._get_default_analysis()
    
    def _analyze_pitch(self, y, sr):
        """Analyze pitch characteristics."""
        # Extract pitch using piptrack
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        
        # Get pitch values with significant magnitude
        pitch_values = []
        for t in range(pitches.shape[1]):
            index = magnitudes[:, t].argmax()
            pitch = pitches[index, t]
            if pitch > 0:
                pitch_values.append(pitch)
        
        if len(pitch_values) == 0:
            return {
                'pitch_variance': 0,
                'pitch_consistency': 0,
                'pitch_range': 0
            }
        
        pitch_values = np.array(pitch_values)
        
        # Calculate pitch statistics
        pitch_variance = np.var(pitch_values)
        pitch_std = np.std(pitch_values)
        pitch_mean = np.mean(pitch_values)
        pitch_range = np.max(pitch_values) - np.min(pitch_values)
        
        # Consistency score (lower variance = more consistent/robotic)
        pitch_consistency = 1.0 / (1.0 + pitch_variance / 1000)
        
        return {
            'pitch_variance': float(pitch_variance),
            'pitch_consistency': float(pitch_consistency),
            'pitch_range': float(pitch_range),
            'pitch_mean': float(pitch_mean),
            'pitch_std': float(pitch_std)
        }
    
    def _analyze_silence(self, y, sr):
        """Analyze silence patterns."""
        # Split audio into frames
        frame_length = 2048
        hop_length = 512
        
        # Calculate RMS energy
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        
        # Define silence threshold (adaptive)
        silence_threshold = np.percentile(rms, 20)
        
        # Find silent frames
        silent_frames = rms < silence_threshold
        silence_ratio = np.mean(silent_frames)
        
        # Analyze silence intervals
        silence_intervals = self._get_intervals(silent_frames)
        
        if len(silence_intervals) > 0:
            avg_silence_duration = np.mean([end - start for start, end in silence_intervals])
            silence_uniformity = 1.0 - np.std([end - start for start, end in silence_intervals]) / (avg_silence_duration + 1e-8)
        else:
            avg_silence_duration = 0
            silence_uniformity = 0
        
        return {
            'silence_ratio': float(silence_ratio),
            'num_silence_intervals': len(silence_intervals),
            'avg_silence_duration': float(avg_silence_duration),
            'silence_uniformity': float(silence_uniformity)
        }
    
    def _analyze_breaths(self, y, sr):
        """Analyze breath-like sounds."""
        # Breaths typically have specific spectral characteristics
        # High frequency content, low amplitude, short duration
        
        # Calculate spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        
        # Calculate zero crossing rate (breaths have higher ZCR)
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        
        # Calculate RMS
        rms = librosa.feature.rms(y=y)[0]
        
        # Detect potential breath events
        # Breaths: high ZCR + low RMS + moderate spectral centroid
        rms_threshold = np.percentile(rms, 30)
        zcr_threshold = np.percentile(zcr, 70)
        
        potential_breaths = (zcr > zcr_threshold) & (rms < rms_threshold)
        
        breath_ratio = np.mean(potential_breaths)
        num_breath_events = len(self._get_intervals(potential_breaths))
        
        return {
            'breath_ratio': float(breath_ratio),
            'num_breath_events': num_breath_events,
            'avg_zcr': float(np.mean(zcr))
        }
    
    def _analyze_spectral(self, y, sr):
        """Analyze spectral characteristics."""
        # Spectral features
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        
        return {
            'spectral_centroid_mean': float(np.mean(spectral_centroid)),
            'spectral_centroid_std': float(np.std(spectral_centroid)),
            'spectral_rolloff_mean': float(np.mean(spectral_rolloff)),
            'spectral_bandwidth_mean': float(np.mean(spectral_bandwidth)),
            'spectral_bandwidth_std': float(np.std(spectral_bandwidth))
        }
    
    def _analyze_energy(self, y, sr):
        """Analyze energy patterns."""
        rms = librosa.feature.rms(y=y)[0]
        
        # Energy statistics
        energy_mean = np.mean(rms)
        energy_std = np.std(rms)
        energy_variance = np.var(rms)
        
        # Energy consistency (AI voices tend to have more consistent energy)
        energy_consistency = 1.0 / (1.0 + energy_variance * 100)
        
        return {
            'energy_mean': float(energy_mean),
            'energy_std': float(energy_std),
            'energy_variance': float(energy_variance),
            'energy_consistency': float(energy_consistency)
        }
    
    def _get_intervals(self, boolean_array):
        """Get start and end indices of True intervals."""
        intervals = []
        in_interval = False
        start = 0
        
        for i, val in enumerate(boolean_array):
            if val and not in_interval:
                start = i
                in_interval = True
            elif not val and in_interval:
                intervals.append((start, i))
                in_interval = False
        
        if in_interval:
            intervals.append((start, len(boolean_array)))
        
        return intervals
    
    def _get_default_analysis(self):
        """Return default analysis in case of error."""
        return {
            'pitch_variance': 0,
            'pitch_consistency': 0,
            'pitch_range': 0,
            'silence_ratio': 0,
            'num_silence_intervals': 0,
            'avg_silence_duration': 0,
            'silence_uniformity': 0,
            'breath_ratio': 0,
            'num_breath_events': 0,
            'avg_zcr': 0,
            'spectral_centroid_mean': 0,
            'energy_consistency': 0
        }


# ============================================================================
# ExplanationGenerator Class (from Block 4)
# ============================================================================

class ExplanationGenerator:
    """
    Generates human-readable explanations based on audio analysis and prediction.
    """
    
    def __init__(self):
        self.analyzer = AudioAnalyzer()
    
    def generate_explanation(self, audio_path, prediction_class, confidence_score):
        """
        Generate explanation based on audio analysis.
        
        Args:
            audio_path: Path to audio file
            prediction_class: 'REAL' or 'AI_GENERATED'
            confidence_score: Confidence score (0-1)
        
        Returns:
            Explanation string
        """
        # Analyze audio
        analysis = self.analyzer.analyze_audio(audio_path)
        
        # Generate explanation based on class and features
        if prediction_class == 'AI_GENERATED':
            explanation = self._explain_ai_generated(analysis, confidence_score)
        else:
            explanation = self._explain_real(analysis, confidence_score)
        
        return explanation
    
    def _explain_ai_generated(self, analysis, confidence):
        """Generate explanation for AI-generated classification."""
        reasons = []
        
        # Check pitch consistency (AI tends to be more consistent)
        if analysis['pitch_consistency'] > 0.7:
            reasons.append("unnatural pitch consistency")
        
        # Check pitch variance (AI tends to have lower variance)
        if analysis['pitch_variance'] < 500:
            reasons.append("limited pitch variation")
        
        # Check silence patterns (AI tends to have uniform silences)
        if analysis['silence_uniformity'] > 0.7:
            reasons.append("uniform silence patterns")
        
        # Check breath events (AI tends to lack natural breaths)
        if analysis['num_breath_events'] < 2:
            reasons.append("absence of natural breath sounds")
        
        # Check energy consistency (AI tends to be more consistent)
        if analysis['energy_consistency'] > 0.7:
            reasons.append("robotic energy patterns")
        
        # Check spectral characteristics
        if analysis['spectral_bandwidth_std'] < 200:
            reasons.append("limited spectral variation")
        
        # Construct explanation
        if len(reasons) == 0:
            explanation = "Artificial speech patterns detected"
        elif len(reasons) == 1:
            explanation = f"{reasons[0].capitalize()} detected"
        elif len(reasons) == 2:
            explanation = f"{reasons[0].capitalize()} and {reasons[1]} detected"
        else:
            explanation = f"{reasons[0].capitalize()}, {reasons[1]}, and {reasons[2]} detected"
        
        return explanation
    
    def _explain_real(self, analysis, confidence):
        """Generate explanation for real voice classification."""
        reasons = []
        
        # Check pitch variation (human tends to have more variation)
        if analysis['pitch_variance'] > 1000:
            reasons.append("natural pitch variation")
        
        # Check breath events (humans have breath sounds)
        if analysis['num_breath_events'] >= 2:
            reasons.append("natural breath patterns")
        
        # Check energy patterns (humans have less consistent energy)
        if analysis['energy_consistency'] < 0.5:
            reasons.append("organic energy fluctuations")
        
        # Check silence patterns (humans have varied pauses)
        if analysis['silence_uniformity'] < 0.5:
            reasons.append("natural pause patterns")
        
        # Check spectral variation
        if analysis['spectral_bandwidth_std'] > 300:
            reasons.append("rich spectral dynamics")
        
        # Construct explanation
        if len(reasons) == 0:
            explanation = "Human speech characteristics detected"
        elif len(reasons) == 1:
            explanation = f"{reasons[0].capitalize()} detected"
        elif len(reasons) == 2:
            explanation = f"{reasons[0].capitalize()} and {reasons[1]} detected"
        else:
            explanation = f"{reasons[0].capitalize()}, {reasons[1]}, and {reasons[2]} detected"
        
        return explanation


# ============================================================================
# LanguageDetector Class
# ============================================================================

class LanguageDetector:
    """
    Detects language from audio file.
    Uses a simple heuristic based on dataset structure.
    For production, you would use a proper language detection model.
    """
    
    def __init__(self):
        # Map language codes to full names
        self.language_map = {
            'ta': 'Tamil',
            'hi': 'Hindi',
            'te': 'Telugu',
            'ml': 'Malayalam',
            'kn': 'Kannada',
            'en': 'English'
        }
    
    def detect_from_filename(self, filename):
        """
        Detect language from filename pattern.
        Assumes filenames contain language codes.
        """
        filename_lower = filename.lower()
        
        # Check for language codes in filename
        for code, language in self.language_map.items():
            if code in filename_lower:
                return language
        
        # Default to English if not detected
        return "English"
    
    def detect_from_audio(self, audio_path):
        """
        Detect language from audio characteristics.
        This is a placeholder - in production, use a proper language detection model.
        """
        # For this implementation, we'll try to detect from filename
        import os
        filename = os.path.basename(audio_path)
        return self.detect_from_filename(filename)


# ============================================================================
# VoiceClassificationPipeline Class
# ============================================================================

class VoiceClassificationPipeline:
    """
    Complete pipeline for voice classification with explanations.
    """
    
    def __init__(self, model_path, normalization_path):
        """
        Initialize the pipeline.
        
        Args:
            model_path: Path to trained model (.h5 file)
            normalization_path: Path to normalization parameters (.npz file)
        """
        print("\n[Initializing Pipeline]")
        
        # Load model
        print("Loading model...")
        self.model = keras.models.load_model(model_path)
        print("✓ Model loaded")
        
        # Load normalization parameters
        print("Loading normalization parameters...")
        norm_params = np.load(normalization_path)
        self.norm_mean = norm_params['mean']
        self.norm_std = norm_params['std']
        print("✓ Normalization parameters loaded")
        
        # Initialize language detector
        self.language_detector = LanguageDetector()
        print("✓ Language detector initialized")
        
        # Initialize explanation generator
        self.explainer = ExplanationGenerator()
        print("✓ Explanation generator initialized")
        
        # Feature extraction parameters (must match training)
        self.sample_rate = 16000
        self.n_lfcc = 40
        self.n_fft = 2048
        self.hop_length = 512
        self.max_duration = 10
        
        print("\n✓ Pipeline ready!")
    
    def extract_features(self, audio_path):
        """Extract LFCC features from audio file."""
        try:
            # Load audio
            y, sr = librosa.load(audio_path, sr=self.sample_rate, duration=self.max_duration)
            
            # Compute LFCC
            lfcc = librosa.feature.mfcc(
                y=y, 
                sr=sr, 
                n_mfcc=self.n_lfcc,
                n_fft=self.n_fft,
                hop_length=self.hop_length
            )
            
            # Standardize to fixed length
            target_length = int(self.max_duration * sr / self.hop_length)
            
            if lfcc.shape[1] < target_length:
                pad_width = target_length - lfcc.shape[1]
                lfcc = np.pad(lfcc, ((0, 0), (0, pad_width)), mode='constant')
            else:
                lfcc = lfcc[:, :target_length]
            
            return lfcc
        
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None
    
    def predict(self, audio_path):
        """
        Make prediction on a single audio file.
        
        Args:
            audio_path: Path to audio file
        
        Returns:
            Dictionary with prediction results
        """
        try:
            # Extract features
            features = self.extract_features(audio_path)
            if features is None:
                return self._get_error_response("Feature extraction failed")
            
            # Reshape and normalize
            features = np.expand_dims(features, axis=0)  # Add batch dimension
            features = np.expand_dims(features, axis=-1)  # Add channel dimension
            features = (features - self.norm_mean) / (self.norm_std + 1e-8)
            
            # Make prediction
            prediction_proba = self.model.predict(features, verbose=0)[0][0]
            
            # Convert to class
            if prediction_proba > 0.5:
                classification = "AI_GENERATED"
                confidence_score = float(prediction_proba)
            else:
                classification = "REAL"
                confidence_score = float(1.0 - prediction_proba)
            
            # Detect language
            language = self.language_detector.detect_from_audio(audio_path)
            
            # Generate explanation
            explanation = self.explainer.generate_explanation(
                audio_path, 
                classification, 
                confidence_score
            )
            
            # Construct response
            response = {
                "status": "success",
                "language": language,
                "classification": classification,
                "confidenceScore": round(confidence_score, 2),
                "explanation": explanation
            }
            
            return response
        
        except Exception as e:
            return self._get_error_response(str(e))
    
    def predict_batch(self, audio_paths):
        """
        Make predictions on multiple audio files.
        
        Args:
            audio_paths: List of audio file paths
        
        Returns:
            List of prediction dictionaries
        """
        results = []
        for audio_path in audio_paths:
            result = self.predict(audio_path)
            results.append(result)
        return results
    
    def _get_error_response(self, error_message):
        """Return error response."""
        return {
            "status": "error",
            "language": "Unknown",
            "classification": "Unknown",
            "confidenceScore": 0.0,
            "explanation": f"Error: {error_message}"
        }


# ============================================================================
# Initialize the pipeline
# ============================================================================

print("\n[Step 1/2] Initializing inference pipeline...")

pipeline = VoiceClassificationPipeline(
    model_path='checkpoints/best_model.h5',
    normalization_path='normalization_params.npz'
)

print("\n[Step 2/2] Pipeline ready for inference!")
print("\nUsage example:")
print("=" * 60)
print("# Single prediction")
print("result = pipeline.predict('path/to/audio.wav')")
print("print(json.dumps(result, indent=2))")
print()
print("# Batch prediction")
print("results = pipeline.predict_batch(['audio1.wav', 'audio2.wav'])")
print("for result in results:")
print("    print(json.dumps(result, indent=2))")
print("=" * 60)

BLOCK 5: INFERENCE PIPELINE

[Step 1/2] Initializing inference pipeline...

[Initializing Pipeline]
Loading model...
✓ Model loaded
Loading normalization parameters...
✓ Normalization parameters loaded
✓ Language detector initialized
✓ Explanation generator initialized

✓ Pipeline ready!

[Step 2/2] Pipeline ready for inference!

Usage example:
# Single prediction
result = pipeline.predict('path/to/audio.wav')
print(json.dumps(result, indent=2))

# Batch prediction
results = pipeline.predict_batch(['audio1.wav', 'audio2.wav'])
for result in results:
    print(json.dumps(result, indent=2))


In [6]:
"""
Block 6: Testing and Evaluation (FIXED - No External Imports)
Test the complete pipeline and generate example predictions.
Run this AFTER running block5_inference_pipeline_fixed.py
"""

import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("BLOCK 6: TESTING AND EVALUATION")
print("=" * 60)

# Check if pipeline exists (from previous block)
try:
    # This should exist if block5 was run
    pipeline
    print("\n✓ Pipeline found from previous block")
except NameError:
    print("\n✗ ERROR: Pipeline not found!")
    print("Please run block5_inference_pipeline_fixed.py first")
    print("\nTo fix this, run:")
    print("  %run block5_inference_pipeline_fixed.py")
    raise SystemExit("Pipeline not initialized")

# Load test data
print("\n[Step 1/5] Loading test dataset...")
test_df = pd.read_csv('test_data.csv')
print(f"Test samples: {len(test_df)}")

# Test on a few samples
print("\n[Step 2/5] Testing on sample files...")
print("\nExample Predictions:")
print("=" * 60)

# Select a few random samples for demonstration
np.random.seed(42)
sample_indices = np.random.choice(len(test_df), size=min(5, len(test_df)), replace=False)

for idx in sample_indices:
    row = test_df.iloc[idx]
    
    print(f"\nSample {idx + 1}:")
    print(f"File: {row['filename']}")
    print(f"True Label: {row['label_text']}")
    
    # Make prediction
    result = pipeline.predict(row['filepath'])
    
    # Display result in JSON format
    print(f"Prediction:")
    print(json.dumps(result, indent=2))
    
    # Check if correct
    is_correct = result['classification'] == row['label_text']
    print(f"Correct: {'✓' if is_correct else '✗'}")
    print("-" * 60)

# Full evaluation on test set
print("\n[Step 3/5] Running full evaluation on test set...")
print("This may take a few minutes...")

all_predictions = []
all_true_labels = []
all_results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
    result = pipeline.predict(row['filepath'])
    
    all_predictions.append(result['classification'])
    all_true_labels.append(row['label_text'])
    all_results.append(result)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Convert to binary (0=REAL, 1=AI_GENERATED)
y_true = [1 if label == 'AI_GENERATED' else 0 for label in all_true_labels]
y_pred = [1 if label == 'AI_GENERATED' else 0 for label in all_predictions]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("\n[Step 4/5] Evaluation Results:")
print("=" * 60)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print("\nConfusion Matrix:")
print("                 Predicted REAL  Predicted AI")
print(f"Actual REAL      {cm[0][0]:14d}  {cm[0][1]:12d}")
print(f"Actual AI        {cm[1][0]:14d}  {cm[1][1]:12d}")

# Analyze confidence scores
all_confidences = [result['confidenceScore'] for result in all_results]
avg_confidence = np.mean(all_confidences)
std_confidence = np.std(all_confidences)

print(f"\nAverage Confidence: {avg_confidence:.4f}")
print(f"Confidence Std Dev: {std_confidence:.4f}")

# Language distribution
language_counts = {}
for result in all_results:
    lang = result['language']
    language_counts[lang] = language_counts.get(lang, 0) + 1

print("\nLanguage Distribution:")
for lang, count in sorted(language_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {lang}: {count} samples ({count/len(all_results)*100:.1f}%)")

# Save detailed results
print("\n[Step 5/5] Saving Results...")

# Save predictions to CSV
results_df = pd.DataFrame({
    'filename': test_df['filename'].values,
    'true_label': all_true_labels,
    'predicted_label': all_predictions,
    'confidence': all_confidences,
    'language': [r['language'] for r in all_results],
    'explanation': [r['explanation'] for r in all_results]
})
results_df.to_csv('test_predictions.csv', index=False)
print("✓ Saved detailed predictions to: test_predictions.csv")

# Save summary metrics
summary = {
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1),
    'avg_confidence': float(avg_confidence),
    'std_confidence': float(std_confidence),
    'confusion_matrix': cm.tolist(),
    'language_distribution': language_counts,
    'total_samples': len(test_df)
}

with open('evaluation_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print("✓ Saved evaluation summary to: evaluation_summary.json")

print("\n" + "=" * 60)
print("EVALUATION COMPLETE!")
print("=" * 60)
print("\nKey Files Generated:")
print("  • test_predictions.csv - Detailed predictions for all test samples")
print("  • evaluation_summary.json - Summary metrics and statistics")
print("\nYou can now use the pipeline for inference on new audio files!")

BLOCK 6: TESTING AND EVALUATION

✓ Pipeline found from previous block

[Step 1/5] Loading test dataset...
Test samples: 27200

[Step 2/5] Testing on sample files...

Example Predictions:

Sample 8923:
File: ma_common_voice_ml_37863448.mp3
True Label: AI_GENERATED
Prediction:
{
  "status": "success",
  "language": "Malayalam",
  "classification": "AI_GENERATED",
  "confidenceScore": 1.0,
  "explanation": "Uniform silence patterns detected"
}
Correct: ✓
------------------------------------------------------------

Sample 25536:
File: ma_mlm_08822_01297943779.mp3
True Label: REAL
Prediction:
{
  "status": "success",
  "language": "Malayalam",
  "classification": "REAL",
  "confidenceScore": 1.0,
  "explanation": "Natural pitch variation, natural breath patterns, and rich spectral dynamics detected"
}
Correct: ✓
------------------------------------------------------------

Sample 21347:
File: ma_mlf_03435_00517990460.mp3
True Label: AI_GENERATED
Prediction:
{
  "status": "success",
  "lang

Processing: 100%|██████████| 27200/27200 [1:21:20<00:00,  5.57it/s]



[Step 4/5] Evaluation Results:
Accuracy:  0.9999 (99.99%)
Precision: 0.9998
Recall:    1.0000
F1-Score:  0.9999

Confusion Matrix:
                 Predicted REAL  Predicted AI
Actual REAL               13597             3
Actual AI                     0         13600

Average Confidence: 0.9999
Confidence Std Dev: 0.0056

Language Distribution:
  English: 7763 samples (28.5%)
  Telugu: 6769 samples (24.9%)
  Hindi: 5175 samples (19.0%)
  Tamil: 4227 samples (15.5%)
  Malayalam: 3266 samples (12.0%)

[Step 5/5] Saving Results...
✓ Saved detailed predictions to: test_predictions.csv
✓ Saved evaluation summary to: evaluation_summary.json

EVALUATION COMPLETE!

Key Files Generated:
  • test_predictions.csv - Detailed predictions for all test samples
  • evaluation_summary.json - Summary metrics and statistics

You can now use the pipeline for inference on new audio files!


In [7]:
"""
Block 6.5 SIMPLE: Quick Model Save
A simpler version if you just want to save the model quickly.
Run this AFTER Block 3 (training) or Block 6 (evaluation).
"""

import os
import shutil
from datetime import datetime
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("BLOCK 6.5: QUICK MODEL SAVE")
print("=" * 60)

# ============================================================================
# Option 1: Save to a timestamped directory
# ============================================================================
print("\n[Option 1: Save with timestamp]")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f'model_saved_{timestamp}'
os.makedirs(save_dir, exist_ok=True)

# Load and save the best model
print("\n[Step 1/3] Loading best model...")
model = keras.models.load_model('checkpoints/best_model.h5')
print("✓ Model loaded")

print("\n[Step 2/3] Saving model...")
model.save(f'{save_dir}/model.h5')
model.save(f'{save_dir}/model.keras')
print(f"✓ Saved: {save_dir}/model.h5")

print("\n[Step 3/3] Copying normalization parameters...")
shutil.copy('normalization_params.npz', f'{save_dir}/normalization_params.npz')
print(f"✓ Saved: {save_dir}/normalization_params.npz")

print(f"\n✓ Model saved to: {save_dir}/")

# ============================================================================
# Option 2: Save to a simple 'final_model' directory
# ============================================================================
print("\n" + "=" * 60)
print("[Option 2: Save to 'final_model' directory]")

final_dir = 'final_model'
os.makedirs(final_dir, exist_ok=True)

print("\n[Step 1/2] Saving model...")
model.save(f'{final_dir}/model.h5')
print(f"✓ Saved: {final_dir}/model.h5")

print("\n[Step 2/2] Copying normalization parameters...")
shutil.copy('normalization_params.npz', f'{final_dir}/normalization_params.npz')
print(f"✓ Saved: {final_dir}/normalization_params.npz")

print(f"\n✓ Model saved to: {final_dir}/")

# ============================================================================
# Option 3: Save additional formats (SavedModel, Weights only)
# ============================================================================
# print("\n" + "=" * 60)
# print("[Option 3: Additional formats (optional)]")

# choice = input("\nSave in additional formats? (y/n): ").strip().lower()

# if choice == 'y':
#     print("\n[Saving TensorFlow SavedModel format...]")
#     model.save(f'{final_dir}/saved_model')
#     print(f"✓ Saved: {final_dir}/saved_model/")
    
#     print("\n[Saving weights only...]")
#     model.save_weights(f'{final_dir}/model_weights.h5')
#     print(f"✓ Saved: {final_dir}/model_weights.h5")
    
#     print("\n[Saving architecture as JSON...]")
#     model_json = model.to_json()
#     with open(f'{final_dir}/model_architecture.json', 'w') as f:
#         f.write(model_json)
#     print(f"✓ Saved: {final_dir}/model_architecture.json")

# ============================================================================
# Summary
# ============================================================================
print("\n" + "=" * 60)
print("SAVE COMPLETE!")
print("=" * 60)

print(f"""
Your model has been saved to TWO locations:

1. Timestamped backup:
   📁 {save_dir}/
      ├── model.h5
      └── normalization_params.npz

2. Final model:
   📁 {final_dir}/
      ├── model.h5
      └── normalization_params.npz
""")

if choice == 'y':
    print(f"""      ├── saved_model/
      ├── model_weights.h5
      └── model_architecture.json
""")

print("""
🎯 To use the saved model:

from tensorflow import keras
import numpy as np

# Load model
model = keras.models.load_model('final_model/model.h5')

# Load normalization params
norm_params = np.load('final_model/normalization_params.npz')
mean = norm_params['mean']
std = norm_params['std']

# Make predictions
# ... (extract features, normalize, predict)
""")

print("\n" + "=" * 60)

# ============================================================================
# Create a simple usage guide
# ============================================================================
usage_guide = f"""# Model Usage Guide

## Saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Files
- `model.h5` - Trained CNN model
- `normalization_params.npz` - Feature normalization parameters

## Quick Usage

### Option 1: Use with inference script
```python
from block7_inference_standalone import VoiceClassifier

classifier = VoiceClassifier(
    model_path='{final_dir}/model.h5',
    norm_path='{final_dir}/normalization_params.npz'
)

result = classifier.classify('audio.wav')
print(result)
```

### Option 2: Manual usage
```python
from tensorflow import keras
import numpy as np
import librosa

# Load model and normalization params
model = keras.models.load_model('{final_dir}/model.h5')
norm_params = np.load('{final_dir}/normalization_params.npz')
mean = norm_params['mean']
std = norm_params['std']

# Extract LFCC features
y, sr = librosa.load('audio.wav', sr=16000, duration=10)
lfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40, n_fft=2048, hop_length=512)

# Pad to fixed length (313 time steps)
target_length = 313
if lfcc.shape[1] < target_length:
    lfcc = np.pad(lfcc, ((0, 0), (0, target_length - lfcc.shape[1])))
else:
    lfcc = lfcc[:, :target_length]

# Reshape and normalize
lfcc = np.expand_dims(lfcc, axis=0)
lfcc = np.expand_dims(lfcc, axis=-1)
lfcc = (lfcc - mean) / (std + 1e-8)

# Predict
prediction = model.predict(lfcc, verbose=0)[0][0]

if prediction > 0.5:
    print(f"AI_GENERATED (confidence: {{prediction:.2f}})")
else:
    print(f"REAL (confidence: {{1-prediction:.2f}})")
```

## Model Specifications
- Input: LFCC features (40 x 313 x 1)
- Output: Binary classification (0 = REAL, 1 = AI_GENERATED)
- Sample Rate: 16000 Hz
- Max Duration: 10 seconds
"""

with open(f'{final_dir}/USAGE.md', 'w') as f:
    f.write(usage_guide)

print(f"✓ Created usage guide: {final_dir}/USAGE.md")
print("=" * 60)

BLOCK 6.5: QUICK MODEL SAVE

[Option 1: Save with timestamp]

[Step 1/3] Loading best model...
✓ Model loaded

[Step 2/3] Saving model...


✓ Saved: model_saved_20260210_045908/model.h5

[Step 3/3] Copying normalization parameters...
✓ Saved: model_saved_20260210_045908/normalization_params.npz

✓ Model saved to: model_saved_20260210_045908/

[Option 2: Save to 'final_model' directory]

[Step 1/2] Saving model...
✓ Saved: final_model/model.h5

[Step 2/2] Copying normalization parameters...
✓ Saved: final_model/normalization_params.npz

✓ Model saved to: final_model/

SAVE COMPLETE!

Your model has been saved to TWO locations:

1. Timestamped backup:
   📁 model_saved_20260210_045908/
      ├── model.h5
      └── normalization_params.npz

2. Final model:
   📁 final_model/
      ├── model.h5
      └── normalization_params.npz



NameError: name 'choice' is not defined

In [ ]:
"""
Block 7: Standalone Inference Script
Ready-to-use script for making predictions on new audio files.
This can be used directly in your hackathon submission.
"""

import numpy as np
import librosa
import json
import argparse
from tensorflow import keras
import warnings
import os
warnings.filterwarnings('ignore')

class AudioAnalyzer:
    """Analyzes audio files to extract interpretable features."""
    
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
    
    def analyze_audio(self, audio_path):
        """Extract audio features for explanation."""
        try:
            y, sr = librosa.load(audio_path, sr=self.sample_rate, duration=10)
            
            # Pitch analysis
            pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
            pitch_values = []
            for t in range(pitches.shape[1]):
                index = magnitudes[:, t].argmax()
                pitch = pitches[index, t]
                if pitch > 0:
                    pitch_values.append(pitch)
            
            pitch_variance = np.var(pitch_values) if len(pitch_values) > 0 else 0
            pitch_consistency = 1.0 / (1.0 + pitch_variance / 1000)
            
            # Silence analysis
            rms = librosa.feature.rms(y=y)[0]
            silence_threshold = np.percentile(rms, 20)
            silent_frames = rms < silence_threshold
            silence_ratio = np.mean(silent_frames)
            
            # Breath analysis
            zcr = librosa.feature.zero_crossing_rate(y)[0]
            rms_threshold = np.percentile(rms, 30)
            zcr_threshold = np.percentile(zcr, 70)
            potential_breaths = (zcr > zcr_threshold) & (rms < rms_threshold)
            breath_ratio = np.mean(potential_breaths)
            num_breath_events = self._count_events(potential_breaths)
            
            # Energy analysis
            energy_variance = np.var(rms)
            energy_consistency = 1.0 / (1.0 + energy_variance * 100)
            
            # Spectral analysis
            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
            spectral_bandwidth_std = np.std(spectral_bandwidth)
            
            return {
                'pitch_variance': float(pitch_variance),
                'pitch_consistency': float(pitch_consistency),
                'silence_ratio': float(silence_ratio),
                'breath_ratio': float(breath_ratio),
                'num_breath_events': num_breath_events,
                'energy_consistency': float(energy_consistency),
                'spectral_bandwidth_std': float(spectral_bandwidth_std)
            }
            
        except Exception as e:
            return self._get_default_analysis()
    
    def _count_events(self, boolean_array):
        """Count number of True intervals."""
        count = 0
        in_interval = False
        for val in boolean_array:
            if val and not in_interval:
                count += 1
                in_interval = True
            elif not val:
                in_interval = False
        return count
    
    def _get_default_analysis(self):
        return {
            'pitch_variance': 0,
            'pitch_consistency': 0,
            'silence_ratio': 0,
            'breath_ratio': 0,
            'num_breath_events': 0,
            'energy_consistency': 0,
            'spectral_bandwidth_std': 0
        }


class ExplanationGenerator:
    """Generates explanations based on audio analysis."""
    
    def __init__(self):
        self.analyzer = AudioAnalyzer()
    
    def generate(self, audio_path, prediction_class, confidence):
        """Generate human-readable explanation."""
        analysis = self.analyzer.analyze_audio(audio_path)
        
        if prediction_class == 'AI_GENERATED':
            return self._explain_ai(analysis)
        else:
            return self._explain_real(analysis)
    
    def _explain_ai(self, analysis):
        reasons = []
        
        if analysis['pitch_consistency'] > 0.7:
            reasons.append("unnatural pitch consistency")
        if analysis['pitch_variance'] < 500:
            reasons.append("limited pitch variation")
        if analysis['num_breath_events'] < 2:
            reasons.append("absence of natural breath sounds")
        if analysis['energy_consistency'] > 0.7:
            reasons.append("robotic speech patterns")
        if analysis['spectral_bandwidth_std'] < 200:
            reasons.append("limited spectral dynamics")
        
        if len(reasons) == 0:
            return "Artificial speech patterns detected"
        elif len(reasons) == 1:
            return f"{reasons[0].capitalize()} detected"
        else:
            return f"{reasons[0].capitalize()} and {reasons[1]} detected"
    
    def _explain_real(self, analysis):
        reasons = []
        
        if analysis['pitch_variance'] > 1000:
            reasons.append("natural pitch variation")
        if analysis['num_breath_events'] >= 2:
            reasons.append("natural breath patterns")
        if analysis['energy_consistency'] < 0.5:
            reasons.append("organic energy fluctuations")
        if analysis['spectral_bandwidth_std'] > 300:
            reasons.append("rich spectral dynamics")
        
        if len(reasons) == 0:
            return "Human speech characteristics detected"
        elif len(reasons) == 1:
            return f"{reasons[0].capitalize()} detected"
        else:
            return f"{reasons[0].capitalize()} and {reasons[1]} detected"


class VoiceClassifier:
    """Main classifier for voice authentication."""
    
    def __init__(self, model_path='checkpoints/best_model.h5', 
                 norm_path='normalization_params.npz'):
        """Initialize classifier with model and parameters."""
        
        # Load model
        self.model = keras.models.load_model(model_path)
        
        # Load normalization parameters
        norm_params = np.load(norm_path)
        self.norm_mean = norm_params['mean']
        self.norm_std = norm_params['std']
        
        # Initialize explainer
        self.explainer = ExplanationGenerator()
        
        # Feature extraction parameters
        self.sample_rate = 16000
        self.n_lfcc = 40
        self.n_fft = 2048
        self.hop_length = 512
        self.max_duration = 10
        
        # Language mapping
        self.language_map = {
            'ta': 'Tamil', 'hi': 'Hindi', 'te': 'Telugu',
            'ml': 'Malayalam', 'kn': 'Kannada', 'en': 'English'
        }
    
    def extract_features(self, audio_path):
        """Extract LFCC features from audio."""
        y, sr = librosa.load(audio_path, sr=self.sample_rate, duration=self.max_duration)
        
        lfcc = librosa.feature.mfcc(
            y=y, sr=sr, n_mfcc=self.n_lfcc,
            n_fft=self.n_fft, hop_length=self.hop_length
        )
        
        target_length = int(self.max_duration * sr / self.hop_length)
        
        if lfcc.shape[1] < target_length:
            pad_width = target_length - lfcc.shape[1]
            lfcc = np.pad(lfcc, ((0, 0), (0, pad_width)), mode='constant')
        else:
            lfcc = lfcc[:, :target_length]
        
        return lfcc
    
    def detect_language(self, filename):
        """Detect language from filename."""
        filename_lower = filename.lower()
        for code, language in self.language_map.items():
            if code in filename_lower:
                return language
        return "English"
    
    def classify(self, audio_path):
        """
        Classify an audio file as REAL or AI_GENERATED.
        
        Returns:
            Dictionary with classification results in the required format
        """
        try:
            # Extract features
            features = self.extract_features(audio_path)
            
            # Reshape and normalize
            features = np.expand_dims(features, axis=0)
            features = np.expand_dims(features, axis=-1)
            features = (features - self.norm_mean) / (self.norm_std + 1e-8)
            
            # Make prediction
            prediction_proba = self.model.predict(features, verbose=0)[0][0]
            
            # Determine class and confidence
            if prediction_proba > 0.5:
                classification = "AI_GENERATED"
                confidence_score = float(prediction_proba)
            else:
                classification = "REAL"
                confidence_score = float(1.0 - prediction_proba)
            
            # Detect language
            filename = os.path.basename(audio_path)
            language = self.detect_language(filename)
            
            # Generate explanation
            explanation = self.explainer.generate(audio_path, classification, confidence_score)
            
            # Return result in required format
            return {
                "status": "success",
                "language": language,
                "classification": classification,
                "confidenceScore": round(confidence_score, 2),
                "explanation": explanation
            }
            
        except Exception as e:
            return {
                "status": "error",
                "language": "Unknown",
                "classification": "Unknown",
                "confidenceScore": 0.0,
                "explanation": f"Error processing audio: {str(e)}"
            }


def main():
    """Main function for command-line usage."""
    parser = argparse.ArgumentParser(description='Voice Classification: AI vs Human')
    parser.add_argument('audio_file', type=str, help='Path to audio file')
    parser.add_argument('--model', type=str, default='checkpoints/best_model.h5',
                       help='Path to model file')
    parser.add_argument('--norm', type=str, default='normalization_params.npz',
                       help='Path to normalization parameters')
    
    args = parser.parse_args()
    
    # Initialize classifier
    print("Initializing classifier...")
    classifier = VoiceClassifier(model_path=args.model, norm_path=args.norm)
    
    # Classify audio
    print(f"Classifying: {args.audio_file}")
    result = classifier.classify(args.audio_file)
    
    # Print result
    print("\n" + "="*60)
    print("CLASSIFICATION RESULT")
    print("="*60)
    print(json.dumps(result, indent=2))
    print("="*60)


if __name__ == '__main__':
    main()